# Spheromones

This is a collection of funny things you can do with/on spheres. It's still work in progress and will likely be expanded slightly over time. If you feel like contributing to it you're very welcome to do so!

In [ ]:
import base64

import geopy
import geojson
import pandas as pd
import numpy as np
import ipyvolume as ipv
import PIL.Image
from ipyleaflet import Map, GeoJSON, Polyline

from utils import extract_coords, extract_lines
from utils import latlon2xyz, lonlat2xyz

## Warm-Up

In [ ]:
ipv.quickscatter(*np.random.random((3, 10_000)), marker="sphere")

In [ ]:
x, y, z = np.random.random((3, 10_000))
color = np.array([x, y, z]).T
ipv.quickscatter(x, y, z, size=z*5, color=color, marker="sphere")

In [ ]:
ipv.figure()
x, y, z = np.random.random((3, 10_000))
scatter = ipv.scatter(x, y, z, marker='sphere')
ipv.show()

In [ ]:
x, y, z = np.random.random((3, 10_000))
scatter.color = np.array([x, y, z]).T
scatter.x = x
scatter.y = y
scatter.z = z

## Random Points

In [ ]:
%load -n latlon2xyz

In [ ]:
lat = 90 * 2 * (np.random.random(10_000) - 0.5)
lon = 180 * 2 * (np.random.random(10_000) - 0.5)
points = [latlon2xyz(*latlon) for latlon in zip(lat, lon)]
x, y, z = np.array(points).T
ipv.quickscatter(x, y, z, size=.5, marker='sphere')

In [ ]:
ipv.figure()
x, y, z = (np.random.random((3, 10_000)) - 0.5) * 2
d = np.sqrt(x**2 + y**2 + z**2)
scatter = ipv.scatter(x/d, y/d , z/d, size=.5, marker='sphere') # box
ipv.xyzlim(-1, 1)
ipv.show()

In [ ]:
d = np.sqrt(x**2 + y**2 + z**2)
scatter.x = x
scatter.y = y
scatter.z = z

In [ ]:
ipv.figure()
for col in 'red blue green'.split():
    x, y, z = (np.random.random((3, 100)) - 0.5)
    d = np.sqrt(x**2 + y**2 + z**2)
    ipv.plot(x/d, y/d , z/d, color=col)
ipv.show()

## Grids

In [ ]:
lon = np.arange(-180, 180, 15)
lat = np.arange(-90, 90 + 15, 15)
points = [latlon2xyz(yi, xi) for xi in lon for yi in lat]
x, y, z = np.array(points).T

ipv.figure()
ipv.scatter(x, y, z, size=2, color='red', marker='sphere')
ipv.xyzlim(-1, 1)
ipv.show()

In [ ]:
delta = 15
lon = np.arange(-180, 180 + delta, delta)
lat = np.arange(-90, 90 + delta, delta)

ipv.figure()
for yi in lat:
    points = [latlon2xyz(yi, xi) for xi in lon]
    x, y, z = np.array(points).T
    ipv.plot(x, y, z, color='red')
for xi in lon:
    points = [latlon2xyz(yi, xi) for yi in lat]
    x, y, z = np.array(points).T
    p = ipv.plot(x, y, z, color='blue')
ipv.show()

## Solid Spheres

Use a ThreeJS primitive

In [ ]:
ipv.figure()
x, y, z = np.array([[0.], [0.], [0.]])
ipv.scatter(x, y, z, size=100, marker="sphere")
ipv.xyzlim(-0.5, 0.5)
ipv.show()

Reimplemented with variable grid:

In [ ]:
def spherical_grid(num_lon, num_lat, r=1):
    "Return array of points on a regular longitude/latitude spherical grid."
    lon = np.linspace(0, 2*np.pi, num_lon)
    lat = np.linspace(-np.pi/2, np.pi/2, num_lat)
    lon, lat = np.meshgrid(lon, lat)
    x = np.ravel(r * np.cos(lat) * np.cos(lon))
    y = np.ravel(r * np.cos(lat) * np.sin(lon))
    z = np.ravel(np.sin(lat))
    return np.array([x, y, z]).T

In [ ]:
def tri(num_lon, num_lat, x):
    "Triangulate spherical grid."
    # FIXME: debug off-by-one issues with num_lon, num_lat
    # FIXME: remove need for x
    l = len(x)
    for j in range(num_lat+1):
        for i in range(num_lon+1):
            a, b, c = [i*num_lat+j, i*num_lat+j+1, i*num_lat+j+num_lon]
            if a < l and b < l and c < l:
                yield [a, b, c]
            a, b, c = [i*num_lat+j+1, i*num_lat+j+num_lon, i*num_lat+j+num_lon+1]
            if a < l and b < l and c < l:
                yield [a, b, c]

In [ ]:
num_lon = 12
num_lat = 13  # FIXME...
ipv.figure()
points = spherical_grid(num_lon=num_lon, num_lat=num_lat)
x, y, z = points.T
triangles = list(tri(num_lon, num_lat, x))
ipv.plot_trisurf(x, y, z, triangles=triangles)
ipv.xyzlim(-1, 1)
ipv.show()

## Thou shall triangulate!

In [ ]:
def triangulate(triangle, level=0):
    "Return a spherical triangle triangulated into four triangles."
    if level == 0:
        yield triangle
    else:
        a, b, c = triangle
        ab2, bc2, ac2 = (a+b)/2, (b+c)/2, (a+c)/2
        ab2, bc2, ac2 = [v / np.linalg.norm(v) for v in [ab2, bc2, ac2]]
        for tri in triangulate([a, ab2, ac2], level-1):
            yield tri
        for tri in triangulate([b, ab2, bc2], level-1):
            yield tri
        for tri in triangulate([c, ac2, bc2], level-1):
            yield tri
        for tri in triangulate([ab2, bc2, ac2], level-1):
            yield tri

In [ ]:
lonlats = [(0, 0), (90, 0), (0, 90)]
points = np.array([lonlat2xyz(lon, lat) for (lon, lat) in lonlats])
ipv.figure()
pts = np.concatenate(list(triangulate(points, level=1)))
ipv.scatter(*pts.T, marker='sphere', size=3)
ipv.scatter(*points, marker='sphere', size=5)
# ipv.plot_trisurf(*pts[-3:], triangles=[[0, 1, 2]])
ipv.xyzlim(-1, 1)
ipv.show()

In [ ]:
lonlats = [(0, 0), (90, 0), (-90, 0), (0, 90), (0, -90), (180, 0)]
points = np.array([lonlat2xyz(lon, lat) for (lon, lat) in lonlats])
surfaces = [
    (0, 1, 3),
    (0, 2, 3),
    (0, 1, 4),
    (0, 2, 4),
    (5, 1, 3),
    (5, 2, 3),
    (5, 1, 4),
    (5, 2, 4),
]

In [ ]:
level = 1
ipv.figure()
for i, j, k in surfaces:
    triangle = points[i], points[j], points[k]
    pts = np.concatenate(list(triangulate(triangle, level=level)))
    x, y, z = pts.T
    ipv.scatter(x, y, z, marker='sphere', size=3)
ipv.xyzlim(-1, 1)
ipv.show()

In [ ]:
level = 1
ipv.figure()
for i, j, k in surfaces:
    triangle = points[i], points[j], points[k]
    pts = np.concatenate(list(triangulate(triangle, level=level)))
    x, y, z = pts.T
    # color = np.array([x, y, z]).T
    # ipv.scatter(x, y, z, marker='sphere', size=3, color=color)
    tris = [list(sub) for sub in np.split(np.arange(len(pts)), len(pts)/3)]
    ipv.plot_trisurf(x, y, z, triangles=tris) # , color=color)
ipv.xyzlim(-1, 1)
ipv.show()

## Globes

Points from CSV 

In [ ]:
df = pd.read_csv('airports.csv', sep=';')

In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
lat = df.latitude_deg.values
lon = df.longitude_deg.values
z = [latlon2xyz(*latlon) for latlon in zip(lat, lon)]
x, y, z = np.array(z).T
ipv.quickscatter(x, y, z, size=.2, color='red', marker='sphere')

GeoJSON on flat maps

In [ ]:
data = geojson.load(open('globe.geojson'))

In [ ]:
m = Map(center=(0, 0), zoom=1)
m

Lines extracted from GeoJSON on a flat map

In [ ]:
g = GeoJSON(data=data, style={'color': 'green'})
m += g

In [ ]:
%load -n extract_lines

In [ ]:
lines = list(extract_lines(data))
for line in lines:
    if len(np.array(line).shape) < 2:
        line = line[0]
    L = len(line)
    if L == 1:
        line = line[0]
    rline = [(lat, lon) for (lon, lat) in line]
    m += Polyline(locations=rline, color='red', fill_color='red',
                  opacity=1, weight=1, dash_array='2', fill_opacity=0.1)

In [ ]:
m -= g

Single points on a globe

In [ ]:
%load -n extract_coords

In [ ]:
ipv.figure()
lon, lat = np.array(list(extract_coords(data))).T
z = [latlon2xyz(yi, xi) for (yi, xi) in zip(lat, lon)]
x, y, z = np.array(z).T
ipv.scatter(x, y, z, size=0.5, color='red', marker='sphere')
ipv.xyzlim(-1, 1)
ipv.show()

In [ ]:
len(x)

Lines extracted from GeoJSON on a globe

In [ ]:
ipv.figure()
x, y, z = np.array([[0.], [0.], [0.]])
ipv.scatter(x, y, z, size=100, color='blue', marker="sphere")
lines = list(extract_lines(data))
for line in lines:
    if len(np.array(line).shape) < 2:
        line = line[0]
    L = len(line)
    if L == 1:
        line = line[0]
    z = [latlon2xyz(lat, lon) for (lon, lat) in line]
    x, y, z = np.array(z).T
    ipv.plot(-x, z, y, color='#aaffaa')
ipv.xyzlim(-1, 1)
ipv.show()

Moving around

In [ ]:
ipv.view(azimuth=-90.0, elevation=10, distance=None)

In [ ]:
def set_lat_lon(lat, lon):
    # x, y, z = np.array(latlon2xyz(lat, lon)).reshape(3, 1)
    # ipv.scatter(x,y, z, size=5, color='red', marker='sphere')
    ipv.view(azimuth=-90 + lon, elevation=lat)

In [ ]:
set_lat_lon(52.3, 14.5)

In [ ]:
def set_location(location: str):
    gc = geopy.geocoders.Nominatim
    loc = gc(user_agent="spheromones").geocode(query=location)
    ipv.view(-90 + loc.longitude, loc.latitude)

In [ ]:
set_location('chicago')

In [ ]:
for lat in np.linspace(0, 90, 1000):
    set_lat_lon(52.3, lat)

Record movie (this needs ImageMagick to be installed!)

In [ ]:
def set_view(figure, framenr, fraction):
    ipv.view(fraction*180, (fraction - 0.5) * 180)

ipv.movie('globe.gif', set_view, fps=20, frames=40, gif_loop=None)

Play recorded movie

In [ ]:
from IPython.display import Image
with open('globe.gif', 'rb') as gif:
    url = b"data:image/gif;base64," + base64.b64encode(gif.read())
Image(url=url.decode('ascii'))

## Textures

Get terrain image to be used as texture image from:

- http://vterrain.org/Imagery/WholeEarth/
- https://eoimages.gsfc.nasa.gov/images/imagerecords/57000/57752/land_shallow_topo_2048.jpg

In [ ]:
image = PIL.Image.open('land_shallow_topo_2048.jpg')

In [ ]:
X = np.arange(-5, 6, 1)
Y = np.arange(-5, 6, 1)
X, Y = np.meshgrid(X, Y)
Z = X * 0 + 1

fig = ipv.figure()
u = (X-5)/10
v = (Y-5)/10
ipv.plot_mesh(X, Y, Z, u=u, v=v, texture=image, wireframe=True)
ipv.show()

In [ ]:
a = np.arange(-1, 1.1, 0.1)
X, Y = np.meshgrid(a, a)
Z = np.sqrt(1**2 - X**2 - Y**2)

ipv.figure()
ipv.plot_surface(X, Z, Y, color="orange")
ipv.plot_wireframe(X, Z, Y, color="red")
ipv.xyzlim(-1, 1)
ipv.show()

In [ ]:
a = np.arange(-1, 1.05, 0.05)
U, V = np.meshgrid(a, a)
X = U
Y = V
Z = np.sqrt(1**2 - X**2 - Y**2)

fig = ipv.figure()
u = (X-1) / 2
v = (Y-1) / 2
mesh = ipv.plot_mesh(X, Y, Z, u=u, v=v, texture=image, wireframe=False)
ipv.xyzlim(-1, 1)
ipv.show()

## More to come... please stay tuned! ;)

- Proper spherical texture images
- Charts
- Animation